# Create a forward operator and display sensitivity maps

`
Authors: Eric Larson <larson.eric.d@gmail.com>
         Denis Engemann <denis.engemann@gmail.com>
         Alex Gramfort <alexandre.gramfort@inria.fr>
`

License: BSD (3-clause)

First setup some paths. We will use the MNE sample data.

In [2]:
%matplotlib qt
import matplotlib.pyplot as plt

import os
import mne

# Change the following path to where the folder ds000117-practical is on your disk
data_path = os.path.expanduser("~/work/data/ds000117-practical/")

raw_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

In [3]:
from IPython.display import Image
from mayavi import mlab

In [4]:
# to check that things work:
#mlab.test_contour3d()
#mlab.show()

# Computing the forward operator

To compute a forward operator we need:

   - a ``-trans.fif`` file that contains the coregistration info
   - a source space
   - the BEM surfaces

## Compute and visualize BEM surfaces

Computing the BEM surfaces requires FreeSurfer and makes use of either of the two following command line tools:

[mne watershed_bem](http://martinos.org/mne/dev/generated/commands.html#mne-watershed-bem)

[mne flash_bem](http://martinos.org/mne/dev/generated/commands.html#mne-flash-bem)

or directly by calling the functions

https://mne.tools/stable/generated/mne.bem.make_watershed_bem.html
https://mne.tools/stable/generated/mne.bem.make_flash_bem.html

Here, we'll assume it's already computed. It takes a few minutes per subject.

So, let's first look at the BEM surfaces.

For EEG we use 3 layers (inner skull, outer skull, and skin), while for MEG 1 layer (inner skull) is enough.

To compute the BEM we'll need to use freesurfer.
You can get it at https://surfer.nmr.mgh.harvard.edu/fswiki/DownloadAndInstall#Download
if you are on mac or linux but you can also use a virtual machine on Windows. See
https://surfer.nmr.mgh.harvard.edu/fswiki/Installation/FreeSurferVirtualImage

To get it in this dataset I did in terminal after setting the SUBJECTS_DIR properly:

`mne watershed_bem -s anat --overwrite`

In [5]:
t1_fname = os.path.join(data_path, 'derivatives/freesurfer/sub-01/ses-mri/anat/mri/T1.mgz')

In [6]:
from nilearn import plotting
plotting.plot_anat(t1_fname);
plt.show()

/Users/claire/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [7]:
subjects_dir = os.path.join(data_path, 'derivatives/freesurfer/sub-01/ses-mri/')

In [8]:
mne.viz.plot_bem(subject='anat', subjects_dir=subjects_dir,
                 orientation='coronal');

Using surface: /Users/claire/work/data/ds000117-practical/derivatives/freesurfer/sub-01/ses-mri/anat/bem/inner_skull.surf
Using surface: /Users/claire/work/data/ds000117-practical/derivatives/freesurfer/sub-01/ses-mri/anat/bem/outer_skull.surf
Using surface: /Users/claire/work/data/ds000117-practical/derivatives/freesurfer/sub-01/ses-mri/anat/bem/outer_skin.surf


In [9]:
#mne.gui.coregistration(subject='anat', subjects_dir=subjects_dir, inst=raw_fname);

### Visualizing the coregistration

The coregistration is the operation that allows to position the head and the sensors in a common coordinate system. In the MNE software, the transformation to align the head and the sensors in stored in a so called *trans* file. It is a FIF file that ends with `-trans.fif`. It can be obtained with ``mne_analyze`` (Unix tools), ``mne.gui.coregistration`` (in Python) or mrilab if you're using a Neuromag system.

For the Python version, see https://mne.tools/dev/generated/mne.gui.coregistration.html

Here, we assume the coregistration is done, so we just visually check the alignment with the following code.

In [10]:
trans_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/anat-trans.fif')
info = mne.io.read_info(raw_fname)
fig = mne.viz.plot_alignment(info, trans_fname, subject='anat', dig=True,
                             subjects_dir=subjects_dir, verbose=True);

Using outer_skin.surf for head surface.
Getting helmet for system 306m


In [11]:
mlab.close('all')

/Users/claire/miniconda3/lib/python3.7/site-packages/mayavi/tools/figure.py:178: UserWarning: Scene all not managed by mlab
  warnings.warn('Scene %s not managed by mlab' % name)


## Compute Source Space

The source space defines the position of the candidate source locations.

The following code computes such a source space with an OCT-6 resolution.

In [12]:
mne.set_log_level('WARNING')
subject = 'anat'
src = mne.setup_source_space(subject, spacing='oct6',
                             subjects_dir=subjects_dir,
                             add_dist=False)

OSError: File "/Users/claire/work/data/ds000117-practical/derivatives/freesurfer/sub-01/ses-mri/anat/surf/lh.sphere" does not exist

In [ ]:
src

``src`` contains two parts, one for the left hemisphere (4098 locations) and one for the right hemisphere (4098 locations).

In [ ]:
fig = mne.viz.plot_alignment(info, trans, subject=subject, dig=False, src=src,
                             subjects_dir=subjects_dir, verbose=True, meg=False, eeg=False);

In [ ]:
mne.viz.plot_alignment(info, trans=trans, subject=subject,
                       src=src, subjects_dir=subjects_dir, dig=True,
                       surfaces=['head-dense', 'white'], coord_frame='meg')

In [ ]:
mlab.close()

### Compute forward solution

We can now compute the forward solution.

To reduce computational load, we'll just compute a single layer BEM
(just inner skull) that can then be used for MEG (not EEG).

In [ ]:
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject=subject, ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

In [16]:
bem_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-bem.fif')
mne.bem.write_bem_solution(bem_fname, bem)

In [ ]:
fwd = mne.make_forward_solution(raw_fname, trans=trans,
                                src=src, bem=bem,
                                meg=True, # include MEG channels
                                eeg=False, # include EEG channels
                                mindist=5.0, # ignore sources <= 5mm from inner skull
                                n_jobs=1) # number of jobs to run in parallel

In [14]:
fwd_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-meg-fwd.fif')
mne.write_forward_solution(fwd_fname, fwd)

Or read the EEG/MEG file from disk

In [ ]:
fwd = mne.read_forward_solution(fwd_fname)

In [ ]:
fwd

Convert to surface orientation for cortically constrained inverse modeling:

In [ ]:
fwd = mne.convert_forward_solution(fwd, surf_ori=True)
leadfield = fwd['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

In [ ]:
fwd

Compute sensitivity maps:

In [ ]:
grad_map = mne.sensitivity_map(fwd, ch_type='grad', mode='fixed')
mag_map = mne.sensitivity_map(fwd, ch_type='mag', mode='fixed')
eeg_map = mne.sensitivity_map(fwd, ch_type='eeg', mode='fixed')

# Show gain matrix a.k.a. leadfield matrix with sensitivy map

In [ ]:
import matplotlib.pyplot as plt

picks_meg = mne.pick_types(fwd['info'], meg=True, eeg=False)
picks_eeg = mne.pick_types(fwd['info'], meg=False, eeg=True)

fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)  
fig.suptitle('Lead field matrix (500 dipoles only)', fontsize=14)

for ax, picks, ch_type in zip(axes, [picks_meg, picks_eeg], ['meg', 'eeg']):
   im = ax.imshow(leadfield[picks, :500], origin='lower', aspect='auto', cmap='RdBu_r')
   ax.set_title(ch_type.upper())
   ax.set_xlabel('sources')
   ax.set_ylabel('sensors')
   plt.colorbar(im, ax=ax, cmap='RdBu_r')

In [ ]:
plt.hist([grad_map.data.ravel(), mag_map.data.ravel(), eeg_map.data.ravel()],
          bins=20, label=['Gradiometers', 'Magnetometers', 'EEG'],
         color=['c', 'b', 'k'])
plt.legend()
plt.title('Normal orientation sensitivity')
plt.xlabel('sensitivity')
plt.ylabel('count');

Uncomment the lines below to view

In [ ]:
# enable correct backend for 3d plotting
clim = dict(kind='percent', lims=(0.0, 50, 99), smoothing_steps=3)  # let's see single dipoles
brain = mag_map.plot(subject='sample', time_label='MAG sensitivity',
                     subjects_dir=subjects_dir, clim=clim, smoothing_steps=8);
view = 'lat'
brain.show_view(view)
brain.save_image('sensitivity_map_grad_%s.jpg' % view)
Image(filename='sensitivity_map_grad_%s.jpg' % view, width=400)

In [ ]:
# enable correct backend for 3d plotting
clim = dict(kind='percent', lims=(0.0, 50, 99), smoothing_steps=3)  # let's see single dipoles
brain = eeg_map.plot(subject='sample', time_label='EEG sensitivity',
                     subjects_dir=subjects_dir, clim=clim, smoothing_steps=8);
view = 'lat'
brain.show_view(view)
brain.save_image('sensitivity_map_eeg_%s.jpg' % view)
Image(filename='sensitivity_map_eeg_%s.jpg' % view, width=400)

## Exercise

Plot the sensitivity maps for EEG and compare it with the MEG. Can you justify the claims that:

- MEG is not sensitive to radial sources
- EEG is more sensitive to deep sources

How will the MEG sensitivity maps and histograms change if you use a free instead of a fixed orientation?

Try changing the `mode` parameter in `mne.sensitivity_map` accordingly.

Why don't we see any dipoles on the gyri?